In [0]:
import os,sys
import h5py
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator,array_to_img, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications
import matplotlib.pyplot as plt
import seaborn as sns
import math
%matplotlib inline
from tqdm import tqdm
from PIL import Image
from google.colab import drive

drive.mount('/content/gdrive')

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


Testing loading from GDrive

In [0]:
with open('/content/gdrive/My Drive/Project/foo.txt', 'w') as f:
  f.write('Hello Google Drive!')
!cat /content/gdrive/My\ Drive/Project/foo.txt

FileNotFoundError: ignored

In [0]:
data_root = "."
with open('/content/gdrive/My Drive/Project/train.csv', 'r') as f:
  train = pd.read_csv(f)
with open('/content/gdrive/My Drive/Project/test.csv', 'r') as f:
  test = pd.read_csv(f)

print(train.label.nunique(),'labels')
print(train.label.nunique(),'labels')
label_counts=train.label.value_counts()
print(label_counts)


In [0]:
plt.figure(figsize = (12,6))
sns.barplot(label_counts.index, label_counts.values, alpha = 0.9)
plt.xticks(rotation = 'vertical')
plt.xlabel('Image Labels', fontsize =12)
plt.ylabel('Counts', fontsize = 12)
plt.show()

In [0]:
for img in tqdm(train.values):
  filename = img[0]
  label = img[1]
  print(filename, label)
  label_dir = ('/content/gdrive/My Drive/Project/train/'+ label)
  destination = ('/content/gdrive/My Drive/Project/train/'+label + '/' + filename + '.jpg')
  im = Image.open('/content/gdrive/My Drive/Project/train_img/'+filename+'.png')
  rgb_im = im.convert('RGB')
  if not os.path.exists('/content/gdrive/My Drive/Project/train/'+ label):
    os.makedirs(label_dir)
  rgb_im.save(destination)
  if not os.path.exists('/content/gdrive/My Drive/Project/train_new1/'+ label):
    os.makedirs('/content/gdrive/My Drive/Project/train_new1/'+ label)
  rgb_im.save('/content/gdrive/My Drive/Project/train_new1/'+ label + '/' + filename + '.jpg')
  

In [0]:
datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

class_size = 600

src_train_dir = ('/content/gdrive/My Drive/Project/train/')
src_dest_dir = ('/content/gdrive/My Drive/Project/train_new1/')
it = 0
for count in label_counts.values:
  ratio = math.floor(class_size/count)-1
  print(count,count*(ratio+1))
  src_lab_dir = ('/content/gdrive/My Drive/Project/train/'+label_counts.index[it])
  dest_lab_dir = ('/content/gdrive/My Drive/Project/train_new1/'+label_counts.index[it])
  if not os.path.exists(dest_lab_dir):
    os.makedirs(dest_lab_dir)
  for file in os.listdir(src_lab_dir):
    img = load_img(os.path.join(src_lab_dir,file))
    x = img_to_array(img)
    x=x.reshape((1,) + x.shape)
    i=0
    for batch in datagen.flow(x, batch_size=1,save_to_dir=dest_lab_dir, save_format='jpg'):
        i+=1
        if i > ratio:
            break 
  it=it+1

NameError: ignored

In [0]:
for dirpath, dirnames, filenames in os.walk('/content/gdrive/My Drive/Project/train_new/'):
    i=0
    label=''
    for filename in [f for f in filenames if f.endswith(".jpg")]:
        label=os.path.split(dirpath)[1]
        i+=1
    print(label,i)

In [7]:
os.chdir('/content/gdrive/My Drive/Project')
!python retrain.py --image_dir train --output_graph model-new-8.pb --output_labels labels-new-8.txt --how_many_training_steps 5000 --learning_rate 0.01 

Instructions for updating:
Use tf.gfile.GFile.
2019-02-20 10:10:24.759104: W tensorflow/core/framework/op_def_util.cc:355] Op BatchNormWithGlobalNormalization is deprecated. It will cease to work in GraphDef version 9. Use tf.nn.batch_normalization().
INFO:tensorflow:Looking for images in 'chips'
INFO:tensorflow:Looking for images in 'rice'
INFO:tensorflow:Looking for images in 'candy'
INFO:tensorflow:Looking for images in 'jam'
INFO:tensorflow:Looking for images in 'coffee'
INFO:tensorflow:Looking for images in 'vinegar'
INFO:tensorflow:Looking for images in 'chocolate'
INFO:tensorflow:Looking for images in 'sugar'
INFO:tensorflow:Looking for images in 'water'
INFO:tensorflow:Looking for images in 'juice'
INFO:tensorflow:Looking for images in 'milk'
INFO:tensorflow:Looking for images in 'soda'
INFO:tensorflow:Looking for images in 'nuts'
INFO:tensorflow:Looking for images in 'spices'
INFO:tensorflow:Looking for images in 'cereal'
INFO:tensorflow:Looking for images in 'beans'
INFO:tens

In [0]:
os.chdir('/tmp')
!rm -r imagenet

In [0]:
!python retrain.py -h

In [0]:
os.chdir('/content/gdrive/My Drive/Project')

In [8]:
IMAGE_SIZE=224
!tflite_convert --graph_def_file=model-new-8.pb --output_file=model-new-8-v1.lite --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE --input_shape=1,299,299,3 --input_array=Mul --output_array=final_result --inference_type=FLOAT --input_data_type=FLOAT --post_training_quantize

2019-02-20 10:19:35.317740: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2299995000 Hz
2019-02-20 10:19:35.318023: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5574b036f760 executing computations on platform Host. Devices:
2019-02-20 10:19:35.318111: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-02-20 10:19:35.413570: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-02-20 10:19:35.414174: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5574b036f080 executing computations on platform CUDA. Devices:
2019-02-20 10:19:35.414228: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla K80, Compute Capability 3.7
2019-02-20 10:19:35.414673: I tensorflow/core/common_runtime/gpu/gpu_device.cc:143

In [0]:
import tensorflow as tf
converter = tf.lite.TFLiteConverter.from_saved_model('model-new-7.pb')
converter.post_training_quantize = True
tflite_quantized_model = converter.convert()
open("quantized_model-new-7.tflite", "wb").write(tflite_quantized_model)

OSError: ignored

In [0]:
IMAGE_SIZE=224
!tflite_convert --graph_def_file=model.pb --output_file=model.lite --input_format=TENSORFLOW_GRAPHDEF --output_format=TFLITE --input_shape=1,${IMAGE_SIZE},${IMAGE_SIZE},3 --input_array=input --output_array=final_result --inference_type=FLOAT --input_data_type=FLOAT

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import argparse
import sys,os
import tensorflow as tf
from PIL import Image
from tqdm import tqdm

In [0]:
t=tqdm(pd.read_csv('test.csv').values)
test=[]
i=0
for tt in t:
    test.append(tt[0])
    i+=1

100%|██████████| 1732/1732 [00:00<00:00, 368945.38it/s]


In [0]:
def load_image(filename):
    #Read in the image_data to be classified."""
    return tf.gfile.FastGFile(filename, 'rb').read()

def load_labels(filename):
    #Read in labels, one label per line."""
    return [line.rstrip() for line in tf.gfile.GFile(filename)]

def load_graph(filename):
    #Unpersists graph from file as default graph."""
    with tf.gfile.FastGFile(filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')

In [0]:
def run_graph(src, dest, labels, input_layer_name, output_layer_name, num_top_predictions):
    with tf.Session() as sess:
        # Feed the image_data as input to the graph.
        # predictions  will contain a two-dimensional array, where one
        # dimension represents the input image count, and the other has
        # predictions per class
        i=0
        with open('submit.csv','w') as outfile:
          for f in os.listdir(src):
              im=Image.open(os.path.join(src,f))
              img=im.convert('RGB')
              img.save(os.path.join(dest,test[i]+'.jpg'))
              image_data=load_image(os.path.join(dest,test[i]+'.jpg'))
              softmax_tensor=sess.graph.get_tensor_by_name(output_layer_name)
              predictions,=sess.run(softmax_tensor, {input_layer_name: image_data})

              # Sort to show labels in order of confidence             
              top_k = predictions.argsort()[-num_top_predictions:][::-1]
              for node_id in top_k:
                  predicted_label = labels[node_id]
                  score = predictions[node_id]
                  print(test[i]+',',predicted_label+","+str(score))
                  outfile.write(test[i]+','+predicted_label+","+str(score)+'\n')
              i+=1

In [0]:
src=os.path.join('test_img')
dest=os.path.join('test_img2')
labels='labels5.txt'
graph='model5.pb'
input_layer='DecodeJpeg/contents:0'
output_layer='final_result:0'
num_top_predictions=1
labels = load_labels(labels)
load_graph(graph)
run_graph(src,dest,labels,input_layer,output_layer,num_top_predictions)

In [0]:
!tflite_convert \
    --output_file=model53.tflite \
    --graph_def_file=model5.pb \
    --input_arrays=Mul \
    --output_arrays=final_result

In [0]:
!lscp